# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

In [94]:
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from tensorflow.python import tf2
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.utils import to_categorical
import numpy as np

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [95]:
# 1
data = pd.read_csv("tic-tac-toe.csv")
data

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [96]:
from sklearn.preprocessing import OneHotEncoder

In [97]:
#2 all are categorical, besides boolean
cat = data.select_dtypes(include='object')
encoder = OneHotEncoder(drop="first").fit(cat) 
encoded = encoder.transform(cat).toarray()
onehot_encoded = pd.DataFrame(encoded)
onehot_encoded 



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0
1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
954,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
955,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0
956,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0


In [98]:
from sklearn.preprocessing import StandardScaler
transformed = StandardScaler().fit_transform(onehot_encoded)
input = pd.DataFrame(transformed, columns=onehot_encoded.columns.tolist())
input

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-0.733294,1.136603,-0.724898,1.238706,-0.733294,1.136603,-0.724898,1.238706,1.348201,-0.957079,1.379504,-0.807294,-0.733294,1.136603,1.379504,-0.807294,1.363709,-0.879815
1,-0.733294,1.136603,-0.724898,1.238706,-0.733294,1.136603,-0.724898,1.238706,1.348201,-0.957079,1.379504,-0.807294,1.363709,-0.879815,-0.724898,1.238706,1.363709,-0.879815
2,-0.733294,1.136603,-0.724898,1.238706,-0.733294,1.136603,-0.724898,1.238706,1.348201,-0.957079,1.379504,-0.807294,1.363709,-0.879815,1.379504,-0.807294,-0.733294,1.136603
3,-0.733294,1.136603,-0.724898,1.238706,-0.733294,1.136603,-0.724898,1.238706,1.348201,-0.957079,1.379504,-0.807294,1.363709,-0.879815,-0.724898,-0.807294,-0.733294,-0.879815
4,-0.733294,1.136603,-0.724898,1.238706,-0.733294,1.136603,-0.724898,1.238706,1.348201,-0.957079,1.379504,-0.807294,-0.733294,-0.879815,1.379504,-0.807294,-0.733294,-0.879815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953,1.363709,-0.879815,-0.724898,1.238706,-0.733294,1.136603,-0.724898,1.238706,1.348201,-0.957079,1.379504,-0.807294,1.363709,-0.879815,-0.724898,1.238706,-0.733294,1.136603
954,1.363709,-0.879815,-0.724898,1.238706,1.363709,-0.879815,-0.724898,1.238706,-0.741729,1.044846,1.379504,-0.807294,-0.733294,1.136603,1.379504,-0.807294,-0.733294,1.136603
955,1.363709,-0.879815,-0.724898,1.238706,1.363709,-0.879815,-0.724898,1.238706,1.348201,-0.957079,-0.724898,1.238706,-0.733294,1.136603,1.379504,-0.807294,-0.733294,1.136603
956,1.363709,-0.879815,-0.724898,1.238706,1.363709,-0.879815,1.379504,-0.807294,-0.741729,1.044846,-0.724898,1.238706,-0.733294,1.136603,1.379504,-0.807294,-0.733294,1.136603


In [99]:
output = data["class"]
output = output.replace({True: 1, False: 0})
output


0      1
1      1
2      1
3      1
4      1
      ..
953    0
954    0
955    0
956    0
957    0
Name: class, Length: 958, dtype: int64

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [100]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(input, output)

In [101]:
from sklearn.datasets import fetch_openml
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes= (30,30), solver='sgd', verbose=1, max_iter=800, random_state=1)

In [102]:
%%time
nn.fit(X_train, y_train)

print("Training set score: %f" % nn.score(X_train, y_train))
print("Test set score: %f" % nn.score(X_test, y_test))



Iteration 1, loss = 0.76571998
Iteration 2, loss = 0.75821735
Iteration 3, loss = 0.74750374
Iteration 4, loss = 0.73643029
Iteration 5, loss = 0.72382205
Iteration 6, loss = 0.71342154
Iteration 7, loss = 0.70289475
Iteration 8, loss = 0.69436888
Iteration 9, loss = 0.68688562
Iteration 10, loss = 0.67994154
Iteration 11, loss = 0.67427753
Iteration 12, loss = 0.66934038
Iteration 13, loss = 0.66519081
Iteration 14, loss = 0.66151821
Iteration 15, loss = 0.65824939
Iteration 16, loss = 0.65508360
Iteration 17, loss = 0.65237673
Iteration 18, loss = 0.65017780
Iteration 19, loss = 0.64770617
Iteration 20, loss = 0.64572898
Iteration 21, loss = 0.64378690
Iteration 22, loss = 0.64198910
Iteration 23, loss = 0.64019379
Iteration 24, loss = 0.63852996
Iteration 25, loss = 0.63681979
Iteration 26, loss = 0.63529819
Iteration 27, loss = 0.63379432
Iteration 28, loss = 0.63238202
Iteration 29, loss = 0.63086178
Iteration 30, loss = 0.62955859
Iteration 31, loss = 0.62820991
Iteration 32, los

/Users/agustinredondoarena/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (800) reached and the optimization hasn't converged yet.
  warnings.warn(


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [103]:
predictions = nn.predict(X_test)
predictions

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1])

In [104]:
test = y_test.values
test

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1])

In [105]:
from sklearn.metrics import mean_squared_error
mean_squared_error(predictions,test)

0.0375

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [106]:
nn = MLPClassifier(hidden_layer_sizes= (30,30), solver='sgd', verbose=1, max_iter=800, random_state=1)
#NOW IT FULFILLS THE EXPECTED RESULTS BUT IT'S OVERFITTED

In [134]:

nn1 = MLPClassifier(hidden_layer_sizes= (5,5), solver='adam', verbose=1, max_iter=40, random_state=1, learning_rate='constant', learning_rate_init=0.09  )

In [135]:
%%time
nn1.fit(X_train, y_train)

print("Training set score: %f" % nn.score(X_train, y_train))
print("Test set score: %f" % nn.score(X_test, y_test))

Iteration 1, loss = 0.64372755
Iteration 2, loss = 0.56107715
Iteration 3, loss = 0.53179644
Iteration 4, loss = 0.48552861
Iteration 5, loss = 0.42656191
Iteration 6, loss = 0.35229676
Iteration 7, loss = 0.26876682
Iteration 8, loss = 0.15886348
Iteration 9, loss = 0.09637851
Iteration 10, loss = 0.08465102
Iteration 11, loss = 0.08354067
Iteration 12, loss = 0.05242449
Iteration 13, loss = 0.04209120
Iteration 14, loss = 0.04603175
Iteration 15, loss = 0.03375288
Iteration 16, loss = 0.04109396
Iteration 17, loss = 0.03681228
Iteration 18, loss = 0.03250354
Iteration 19, loss = 0.02766174
Iteration 20, loss = 0.03163326
Iteration 21, loss = 0.03710721
Iteration 22, loss = 0.02557057
Iteration 23, loss = 0.02281638
Iteration 24, loss = 0.02961234
Iteration 25, loss = 0.02108522
Iteration 26, loss = 0.02784655
Iteration 27, loss = 0.02638034
Iteration 28, loss = 0.01729666
Iteration 29, loss = 0.02354608
Iteration 30, loss = 0.01520294
Iteration 31, loss = 0.01563398
Iteration 32, los

/Users/agustinredondoarena/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (40) reached and the optimization hasn't converged yet.
  warnings.warn(


In [136]:
predictions1 = nn1.predict(X_test)
mean_squared_error(predictions1,test)

0.025

**Which approach(es) did you find helpful to improve your model performance?**

In [137]:
# THE LOSS WAS IMPROVED WITH LESS ITERATIONS